# Calculate Stints for Contributors

In [37]:
from collections import Counter
from django.core.exceptions import ObjectDoesNotExist
# Get a set of bylines
def get_bylines(contributor):
    return contributor.byline_set.order_by('story__publication_date')

class Contributions:
    
    def __init__(self, contributor):
        self.contributor = contributor
        self.bylines = contributor.byline_set.exclude(
            story__publication_date=None).order_by('story__publication_date')
        self.in_house_bylines = self.bylines.exclude(story__story_type__section__title='Debatt')
        self.sections = Counter(
            self.bylines.values_list('story__story_type__section__title', flat=True)
            )
        self.credits = Counter(
            self.bylines.values_list('credit', flat=True)
        )
        
    @property
    def is_external(self):
        return self.in_house_bylines.count() == 0
    
    def create_stints(self):
        journalist_stint = self._create_stint('journalist', ['text', 'by', 'text and photo'])
        photographter_stint = self._create_stint('fotograf', ['photo', 'text and photo'])
        illustrator_stint = self._create_stint('illustratør', ['illustration'])
        return self.stints
        
    @property 
    def stints(self):
        return Stint.objects.filter(contributor=self.contributor)
    
    def _create_stint(self, position_title, byline_types):
        bylines = self.in_house_bylines.filter(credit__in=byline_types)
        position = Position.objects.get_or_create(title=position_title)[0]
        if bylines:
            stint = Stint.objects.get_or_create(contributor=self.contributor, position=position)[0]
            stint.start_date = bylines.first().story.publication_date.date()
            stint.end_date = bylines.last().story.publication_date.date()
            stint.save()
            return stint
        
    def get_status(self, save=False):
        cutoff = timezone.timedelta(days=30*6) 
        if self.is_external:
            status = Contributor.EXTERNAL
        else:
            try:
                last_contribution = self.bylines.last().story.publication_date 
            except AttributeError:
                status = Contributor.UNKNOWN 
            else:
                if timezone.now() - last_contribution > cutoff:
                    status = Contributor.RETIRED
                else:
                    status = Contributor.ACTIVE
        if save:
            self.contributor.status = status
            self.contributor.save()
        
        return status
        
        
            
mari = Contributor.objects.first()
maris_contribs = Contributions(mari)
maris_contribs.get_status(save=True)
mari.get_status_display()

'Slutta'

In [38]:
maris_contribs.stints

[<Stint: journalist Mari Lund Haanshus (2007-02-20–2010-03-09)>, <Stint: fotograf Mari Lund Haanshus (2007-09-25–2010-03-09)>, <Stint: illustratør Mari Lund Haanshus (2009-03-24–2009-05-26)>]

In [39]:
maris_contribs.create_stints()

[<Stint: journalist Mari Lund Haanshus (2007-02-20–2010-03-09)>, <Stint: fotograf Mari Lund Haanshus (2007-09-25–2010-03-09)>, <Stint: illustratør Mari Lund Haanshus (2009-03-24–2009-05-26)>]

In [34]:
maris_contribs.credits

Counter({'by': 28,
         'illustration': 2,
         'photo': 38,
         'text': 26,
         'text and photo': 16})

In [40]:
for contributor in Contributor.objects.order_by('?'):
    contr = Contributions(contributor)
    contr.create_stints()
    contr.get_status(save=True)
    print('{name} {status}, {stints}'.format(
        name=contributor.display_name,
        status=contributor.get_status_display(),
        stints=','.join(str(stint) for stint in contr.stints),
        ))

Stortingsrepresentanter Ekstern, 
Jon A. Evang Ekstern, 
Ingvild Rønneberg Hidle Ekstern, 
Maryam Iqbal Tahir Ekstern, 
Eirik Birkeland Ekstern, 
Ingeborg Huse Amundsen Slutta, journalist Ingeborg Huse Amundsen (2011-02-15–2012-05-22)
Knut Sørsdal Ekstern, 
Heimkjær Vestlending Slutta, journalist Heimkjær Vestlending (2009-11-10–2009-11-10)
Julie Slutta, journalist Julie (2004-09-28–2004-09-28)
Egil Ellenes Slutta, journalist Egil Ellenes (2005-10-11–2005-11-01)
Jølle Slutta, journalist Jølle (2008-08-19–2009-04-28)
Andreas Johannesen Delsett Ekstern, 
Pål Henriksen Ekstern, 
Gunn Elisabeth Birkelund Ekstern, 
Christian A. Drevon Ekstern, 
Anne Raaum Christensen Ekstern, 
Ingrid Tungen Ekstern, 
Katharina Simonsen Slutta, journalist Katharina Simonsen (2009-09-08–2010-11-23)
Det Humanistiske Fakultets Studentutvalg Ekstern, 
Kaja De Ru Ekstern, 
Kristine Gramstad Slutta, fotograf Kristine Gramstad (2004-09-07–2005-08-16),journalist Kristine Gramstad (2004-09-28–2005-05-31)
Rolf Mikkels

In [36]:
%debug

> <ipython-input-31-6407969bac9e>(39)_create_stint()
     38             stint = Stint.objects.get_or_create(contributor=self.contributor, position=position)[0]
---> 39             stint.start_date = bylines.first().story.publication_date.date()
     40             stint.end_date = bylines.last().story.publication_date.date()

ipdb> bylines
[<Byline: Tekst: Birk Tjeldflaat Helle (Nå saksøker han SiO)>]
ipdb> bylines.first().story
<Story: Nå saksøker han SiO>
ipdb> bylines.first().story.publication_date
ipdb> q
